In [ ]:
from HDF.HDFData import HDFArray
import matplotlib.pyplot as plt
import peakdetect as pkd
import os.path
import math
from Analyzer import *
from config import settings
path = settings["paths"]["filtered"]
count_files = 85

In [ ]:
#Найдем минимальную длину интервала, которая подойдет для точного прогнозирования данных
file = FileAnalyzer(path + '0.h5')
length_min_interval = file.get_min_length_intervals()
length_min_intervals_from_files = []
for i in range(1, count_files):
    filename = path + str(i) + '.h5'
    file = FileAnalyzer(filename)
    min_length = file.get_min_length_intervals()
    length_min_intervals_from_files.append(min_length)
    print('file = ', filename, ' with min length interval = ', min_length)
    if file.get_min_length_intervals() < length_min_interval:
        length_min_interval = file.get_min_length_intervals()
print('Min length interval = ', length_min_interval)

In [ ]:
plt.plot(length_min_intervals_from_files)
#График минимальных длин интервалов из полученных файлов

In [ ]:
average_series = sum(length_min_intervals_from_files)/count_files
print('Среднее полученного ряда: ', average_series)

In [ ]:
count = 0
for i in range(count_files):
    filename = path + str(i) + '.h5'
    file = FileAnalyzer(filename)
    min_length = file.get_min_length_intervals()
    if min_length > average_series:
        count+=1
print('count files = ', count)
#Получили 33 файла из 85, что не очень положительно влияет на обучающую выборку
#Попробуем взять медиану ряда

In [ ]:
median_of_series = sorted(length_min_intervals_from_files)[42] # Взяли медиану ряда
print('Медиана полученного ряда: ', median_of_series)

In [ ]:
count = 0
good_files = []
for i in range(count_files):
    filename = path + str(i) + '.h5'
    file = FileAnalyzer(filename)
    min_length = file.get_min_length_intervals()
    if min_length > median_of_series and file.is_good_file():
        good_files.append(str(i)+'.h5')
print('Count good files = ', len(good_files))
#Здесь уже видим, что нам подходят 42 файла с минимальной длинной интервала 66.
#Также три файла говорят о некорректности данных, хранящихся в них.

In [ ]:
print(good_files)

In [ ]:
count_all_intervals = 0
for filename in good_files:
    file = FileAnalyzer(path + filename)
    count_all_intervals += len(file.get_intervals())
print('Всего интервалов во всех подходящих файлах: ', count_all_intervals)

In [ ]:
count_files_for_train = int(len(good_files) * 0.9)
count_files_for_test = len(good_files) - count_files_for_train
print('Файлов для обучения: ', count_files_for_train)
print('Файлов для проверки: ', count_files_for_test)

In [ ]:
# Сделаем датасет train
import csv
import struct
path_to_train = settings["paths"]["train"]
id_file = 0
y = []
id_begin_interval = 0
for i in range(count_files_for_train):
    filename = good_files[i]
    file = FileAnalyzer(path + filename)
    count_intervals = len(file.get_intervals())
    if (len(y) == 0):
        res = file.trim_intervals_and_save(path_to_train,id_begin_interval, median_of_series)
        y = res
        id_begin_interval += len(y)
    else:
        res = file.trim_intervals_and_save(path_to_train,id_begin_interval, median_of_series)
        y = np.vstack((y, res))
        id_begin_interval += len(y)
    id_file += 1

filename = path_to_train +'y.csv'
with open(filename, "w", newline='') as out_file:
    writer = csv.writer(out_file, delimiter=';')
    writer.writerows(y)





ERROR len track =  1  len interval =  10000000000.0
